In [1]:
import attr
import numpy as np
import xsimlab as xs


So it is very important, that the index is variable, and can vary between created instances of class
# I extended the code example below to show what still does not work: variable dimensions

I think the only way this can actually work, is if either the handling of write_index_vars() function within xsimlab/drivers.py is modified, or the actual index variable name is changed by the make_process, not just the dims. I will experiment some more, any help will be very much appreciated.


In [3]:
# do not decorate A with attr.s or xs.process!!

class A:
    x = xs.index(dims='x')
    
    dim = xs.variable(intent='in')

    def initialize(self):
        self.x = np.arange(self.dim)

In [4]:
def make_process(cls_name, dim):
    new_cls = type(cls_name, A.__bases__, dict(A.__dict__))
    
    new_cls.x.metadata['dims'] = dim
    
    return xs.process(new_cls)


In [5]:
B = make_process('B', 'y')

In [6]:
B

__main__.B

In [7]:
xs.process_info(B)

<B  (xsimlab process)>
Variables:
    x          [out] y 
    dim         [in]
Simulation stages:
    initialize


In [8]:
C = make_process('C', 'z')

In [9]:
C

__main__.C

In [10]:
xs.process_info(C)

<C  (xsimlab process)>
Variables:
    x          [out] z 
    dim         [in]
Simulation stages:
    initialize


In [11]:

Model = xs.Model({'C':C,'B':B})

In [12]:
in_DS = xs.create_setup(model=Model,
                           # necessary for xsimlab
                           clocks={'clock': [0, 1]},
                           input_vars={
                               'C__dim':1,
                               'B__dim':2
                           }
                       )

In [13]:
with Model:
    out_DS = in_DS.xsimlab.run()

ValueError: parameter 'value': expected array with shape (1,), got (2,)

In [14]:
out_DS

NameError: name 'out_DS' is not defined